In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import display


In [2]:
def highlight_atoms(mol, highlight_dict):
    """
    Met en évidence des atomes spécifiques dans une molécule avec des couleurs spécifiées.
    
    Parameters:
    mol (rdkit.Chem.Mol): Molécule RDKit.
    highlight_dict (dict): Dictionnaire avec les indices des atomes comme clés et les couleurs comme valeurs.
    
    Returns:
    PIL.Image.Image: Image de la molécule avec les atomes mis en évidence.
    """
    drawer = Draw.MolDraw2DCairo(300, 300)
    atom_colors = {idx: color for idx, color in highlight_dict.items()}
    atom_indices = list(highlight_dict.keys())
    Draw.rdMolDraw2D.PrepareAndDrawMolecule(drawer, mol, highlightAtoms=atom_indices, highlightAtomColors=atom_colors)
    drawer.FinishDrawing()
    return drawer.GetDrawingText()

In [3]:
# Fonction pour visualiser les molécules d'une certaine catégorie de crèmes
def visualize_molecules_for_cream(database, cream_name):
    """
    Affiche les molécules d'une certaine catégorie de crèmes avec des atomes spécifiques mis en évidence.
    
    Parameters:
    database (str): Chemin vers le fichier CSV de la base de données.
    cream_name (str): Nom de la crème à visualiser.
    """
    # Filtrer les molécules pour la catégorie de crème spécifiée
    filtered_df = df[df['Cream'] == cream_name]
    
    # Couleurs pour les atomes spécifiques
    color_map = {
        'Isopropyl': (0, 1, 0),  # Vert
        'NaOH': (0, 0, 1),       # Bleu
        'Linalool': (1, 0.5, 0), # Orange 1
        'Citronellol': (1, 0.6, 0), # Orange 2
        'Limonene': (1, 0.7, 0),   # Orange 3
        'Benzyl Alcohol': (1, 0.8, 0), # Orange 4
        'Benzyl Salicylate': (1, 0.9, 0) # Orange 5
    }
    
    # Motifs SMARTS pour les composés spécifiques
    smarts_patterns = {
        'Isopropyl': 'CC(C)O',
        'NaOH': '[Na+].[OH-]',
        'Linalool': 'CC(C)CC1=CC=C(C=C1)O',
        'Citronellol': 'CC(C)CCC(C)(C)CO',
        'Limonene': 'CC1=CC=CCC1(C)C',
        'Benzyl Alcohol': 'C1=CC=C(C=C1)CO',
        'Benzyl Salicylate': 'C1=CC=C(C=C1)OC(=O)C2=CC=CC=C2'
    }
    
    # Parcourir les molécules filtrées et afficher les images avec les atomes mis en évidence
    for index, row in filtered_df.iterrows():
        smi = row['SMILES']
        mol = Chem.MolFromSmiles(smi)
        
        # Identifier les atomes à mettre en évidence
        highlight_dict = {}
        for compound, smarts in smarts_patterns.items():
            pattern = Chem.MolFromSmarts(smarts)
            matches = mol.GetSubstructMatches(pattern)
            for match in matches:
                for idx in match:
                    highlight_dict[idx] = color_map[compound]
        
        # Générer l'image de la molécule avec les atomes mis en évidence
        if mol:
            img_data = highlight_atoms(mol, highlight_dict)
            display(Draw.MolToImage(mol, highlightAtoms=highlight_dict.keys(), highlightAtomColors=highlight_dict))
            with open(f"molecule_{index}.png", "wb") as f:
                f.write(img_data)

# Exemple d'utilisation
#database_path = 'path/to/your/database.csv'  # Remplacez par le chemin correct de votre fichier CSV
#cream_name = 'Nuxe'  # Remplacez par le nom de la crème que vous souhaitez visualiser
#visualize_molecules_for_cream(database_path, cream_name)